In [11]:
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 환경 변수 사용
db_host = os.getenv("DATABASE_HOST")
db_name = os.getenv("DATABASE_NAME")
db_user = os.getenv("DATABASE_USER")
db_password = os.getenv("DATABASE_PASSWORD")

In [14]:
import random

class RandomNicknameGenerator:
    # 형용사 리스트
    adjectives = [
        "배부른", "행복한", "용감한", "활기찬", "지혜로운",
        "즐거운", "슬기로운", "긍정적인", "평화로운", "창의적인"
    ]

    # 명사 리스트
    nouns = [
        "토끼", "사자", "호랑이", "고양이", "강아지",
        "여우", "나무늘보", "팬더", "코알라", "펭귄",
        "햄스터", "다람쥐", "앵무새", "알파카", "쿼카"
    ]

    @staticmethod
    def generate_nickname():
        # 형용사와 명사 리스트에서 랜덤으로 하나씩 선택
        adjective = random.choice(RandomNicknameGenerator.adjectives)
        noun = random.choice(RandomNicknameGenerator.nouns)

        # 0에서 9999 사이의 랜덤 숫자 생성
        number = random.randint(0, 9999)

        # 닉네임 조합
        return f"{adjective} {noun} {number}"

print(RandomNicknameGenerator.generate_nickname())

슬기로운 팬더 230


In [12]:
import psycopg2

connection = psycopg2.connect(
    host=db_host,
    database=db_name,
    user=db_user,
    password=db_password,
    port="5432"  # 기본 포트
)

print (connection)
cursor = connection.cursor()

# 데이터베이스 작업 수행
cursor.execute("SELECT * FROM \"CulturePost\";")
results = cursor.fetchall()
for row in results:
    print(row)

cursor.close()
connection.close()

<connection object at 0x1073ed6d0; dsn: 'user=postgres password=xxx dbname=reculture host=35.226.243.3 port=5432', closed: 0>
(1, '파일럿', '✈️', datetime.datetime(2024, 8, 20, 16, 13, 53, 726000), 'PUBLIC', '파일럿', '조정식, 이주명, 한선화 출연 / 김한결 감독', '코미디', '', 1, 1, '역시 믿고 보는 코미디 영화!', datetime.datetime(2024, 8, 13, 15, 9, 10, 606000), datetime.datetime(2024, 8, 20, 16, 14, 55, 735000))
(4, '8/21 movie', '\U0001f979', datetime.datetime(2024, 8, 20, 17, 18, 56), 'PRIVATE', 'Troll', 'Idk', 'Animation', '', 2, 1, 'Cool', datetime.datetime(2024, 8, 20, 17, 20, 23, 393000), datetime.datetime(2024, 8, 20, 17, 20, 23, 393000))
(5, '8/15 musical', '🥲', datetime.datetime(2024, 8, 15, 5, 23, 53), 'PRIVATE', '영웅', 'somewhere', 'who', '', 2, 2, 'nice', datetime.datetime(2024, 8, 21, 5, 25, 1, 495000), datetime.datetime(2024, 8, 21, 5, 25, 1, 495000))
(6, '8/15 Book', '😗', datetime.datetime(2024, 8, 15, 5, 25, 38), 'PUBLIC', 'Book', 'Me', '7.15~8/15', 'Gg', 2, 7, 'Ggg', datetime.datetime(2024, 8, 21, 5, 26,